In [1]:
from data_ingestion.vector_db import get_vector_store
from data_ingestion.utils.utils import get_website_html
from data_ingestion.scraper import get_articles_publication_dates, extract_images_urls

In [2]:
THE_BATCH_URL = "https://www.deeplearning.ai/the-batch/"

base_url_html = get_website_html(THE_BATCH_URL)
articles_dates = get_articles_publication_dates(base_url_html)
print(articles_dates[:5])

['Jan 01, 2025', 'Dec 25, 2024', 'Dec 18, 2024', 'Dec 11, 2024', 'Dec 04, 2024']


In [3]:
def process_vector_store_metadatas(vector_store):
    vector_db = vector_store.get()
    metadatas = vector_db['metadatas']
    articles_metadatas = []
    for metadata in metadatas:
        if metadata['source'] == 0:
            articles_metadatas.append(metadata)

    articles_metadatas = sorted(articles_metadatas, key = lambda x: x['article_link'])
    articles_metadatas.reverse()
    articles_metadatas = [{k: v for k, v in met.items() if k != "source" and k != "chunk_heading"} for met in articles_metadatas]

    articles_dates = get_articles_publication_dates(base_url_html)
    
    dates_metadatas = []
    for metadata in zip(articles_metadatas, articles_dates):
        new_metadata = metadata[0]
        new_metadata["article_publication_date"] = metadata[1]
        dates_metadatas.append(new_metadata)

    image_urls = extract_images_urls(base_url_html)

    final_metadata = []
    for met in zip(dates_metadatas, image_urls):
        met[0]["article_image_url"] = met[1]
        final_metadata.append(met[0])

    return final_metadata

In [4]:
vector_store = get_vector_store()
metadatas = process_vector_store_metadatas(vector_store)
metadatas

Featured image URL: https://dl-staging-website.ghost.io/content/images/2025/01/unnamed--35--1.png


[{'article_link': 'https://www.deeplearning.ai/the-batch/issue-282/',
  'article_title': 'Happy New Year! Hopes For 2025 With Mustafa Suleyman, Audrey Tang, Albert Gu, Hanno Basse, Joseph Gonzalez, David Ding',
  'article_publication_date': 'Jan 01, 2025',
  'article_image_url': 'https://dl-staging-website.ghost.io/content/images/2025/01/unnamed--35--1.png'},
 {'article_link': 'https://www.deeplearning.ai/the-batch/issue-281/',
  'article_title': 'Top AI Stories of 2024! Agents Rise, Prices Fall, Models Shrink, Video Takes Off, Acquisitions Morph',
  'article_publication_date': 'Dec 25, 2024',
  'article_image_url': 'https://dl-staging-website.ghost.io/content/images/2024/12/unnamed--39--1.jpg'},
 {'article_link': 'https://www.deeplearning.ai/the-batch/issue-280/',
  'article_title': 'Phi-4 Breaks Size Barrier, HunyuanVideo Narrows Open Source Gap, Gemini 2.0 Flash Accelerates Multimodal Modeling, LLMs Propose Research Ideas',
  'article_publication_date': 'Dec 18, 2024',
  'article_im